In [14]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

df = pd.read_csv(r"C:\ise3230\nutrition_and_price_dataset.csv")

items = df["food_name"].tolist()

fat = df.set_index("food_name")["fat_g"].to_dict()
fiber = df.set_index("food_name")["fiber_g"].to_dict()
sugar = df.set_index("food_name")["sugar_g"].to_dict()
sodium = df.set_index("food_name")["sodium_mg"].to_dict()
protein = df.set_index("food_name")["protein_g"].to_dict()
calories = df.set_index("food_name")["calories_kcal"].to_dict()
cost = df.set_index("food_name")["price_per_serving_usd"].to_dict()

m = gp.Model("diet")
m.Params.LogToConsole = 0

# X represents the count of each food
x = m.addVars(items, vtype=GRB.INTEGER, lb=0, name="x")
# Y is a binary variable reprensenting whether we're using each food
# Y is used to enforce the constraint of having atleast 10 unique items
y = m.addVars(items, vtype=GRB.BINARY, name="y")

m.setObjective(gp.quicksum(cost[i] * x[i] for i in items), GRB.MINIMIZE)

FAT_MAX = 275
SODIUM_MAX = 6000
PROTEIN_MIN = 150
CALORIE_MIN = 10000
FIBER_MIN = 75
SUGAR_MAX = 100

# Nutrition constraints
m.addConstr(gp.quicksum(fat[i] * x[i] for i in items) <= FAT_MAX, "fat_max")
m.addConstr(gp.quicksum(sodium[i] * x[i] for i in items) <= SODIUM_MAX, "sodium_max")
m.addConstr(gp.quicksum(protein[i] *  x[i] for i in items) >= PROTEIN_MIN, "protein_min")
m.addConstr(gp.quicksum(calories[i] * x[i] for i in items) >= CALORIE_MIN, "calorie_min")
m.addConstr(gp.quicksum(fiber[i] * x[i] for i in items) >= FIBER_MIN, "fiber_min")
m.addConstr(gp.quicksum(sugar[i] * x[i] for i in items) <= SUGAR_MAX, "sugar_max")
m.addConstr(gp.quicksum(x[i] for i in items) <= 30, "max_items")

# More than 10 unique items constraints
M = 1000
for i in items:
    m.addConstr(x[i] <= M * y[i], name=f"min_items_constraint1")
    m.addConstr(x[i] >= y[i], name=f"min_items_constraint2")

m.addConstr(gp.quicksum(y[i] for i in items) >= 10, "minimum_distinct_items")

m.optimize()

if m.Status == GRB.OPTIMAL:
    print("obj =", m.ObjVal)
    for i in items:
        if x[i].X > 1e-6:
            print(i, int(round(x[i].X)))
else:
    print("Status:", m.Status)

Set parameter LogToConsole to value 0


obj = 25.47
Corn dog 1
Large pepperoni pizza (2 slices) 3
Instant oatmeal packet 18
Lentils (cooked) 1
Cereal (bran) 1
English muffin 2
Granola 1
Mashed potatoes 1
Tortilla (flour) 1
Whole wheat bread 1
